In [ ]:
from src.data import DocumentWarehouse, NERDataset
from transformers import LayoutLMv2Processor
import yaml

In [ ]:
with open('../conf/dataset/FUNSD.yaml') as f: # will be replace with hydra config
    label2id = yaml.load(f, Loader=yaml.FullLoader)['label2id']

In [ ]:
processor = LayoutLMv2Processor.from_pretrained("microsoft/layoutlmv2-base-uncased",
                                                   revision="no_ocr")

In [ ]:
dataset = DocumentWarehouse("../../datasets/FUNSD/images","../../datasets/FUNSD/annotations").get_datasets()

In [ ]:
NERDataset(dataset['train_dataset'],processor,label2id).__getitem__(0)